# OLMoE Routing Experiments - Complete End-to-End Notebook

**Complete workflow from installation to results analysis**

This notebook runs on:
- ✅ **Google Colab** (Recommended - GPU enabled)
- ✅ Local Jupyter (GPU or CPU)
- ✅ Kaggle, Paperspace, etc.

---

## 🚀 Quick Start (Google Colab)

### Recommended: Run in Google Drive
1. Upload this notebook to Google Drive
2. Open with Google Colab
3. Enable GPU: `Runtime → Change runtime type → GPU`
4. Run all cells - the notebook will:
   - Mount your Google Drive
   - Clone/update the GitHub repository to Drive
   - Install dependencies
   - Run experiments with **internal routing logs**
   - Save results to Drive

**Benefits:**
- Repository persists in Drive (only clone once!)
- Results saved to Drive automatically
- **Internal router_logits logged for every sample**
- Faster subsequent runs (repo already available)

---

## 📋 What This Notebook Does

1. **Environment Detection**: Auto-detects Colab/Kaggle/local
2. **Repository Setup**: Clones repo to Google Drive (or uses existing)
3. **GPU Configuration**: Configures GPU/CPU automatically
4. **Installation**: Installs all required packages
5. **Custom Routing**: Implements custom expert selection with internal logging
6. **Internal Logs Demo**: Shows exactly what gets logged
7. **Experiments**: Tests different routing strategies and expert counts
8. **Two-Phase Analysis**: Analyzes baseline, then tests modified routing
9. **Complete Visibility**: Access router_logits for every decision
10. **Comprehensive Reports**: Generates visualizations and detailed reports

---

## 📁 File Locations (Google Drive)

When running in Colab, files are stored in:
- **Repository**: `/content/drive/MyDrive/MOE-with-feature-selection/`
- **Results**: `/content/drive/MyDrive/olmoe_experiments/`
- **Internal Logs**: `olmoe_experiments/*/logs/*_internal_routing.json`

Your results AND internal logs persist across Colab sessions!

---

## Table of Contents

1. [Environment Setup](#1-environment-setup)
2. [GPU Configuration](#2-gpu-configuration)
3. [Installation](#3-installation)
4. [Custom Expert Selection & Model Patching](#4-custom-expert-selection--model-patching)
   - **4.5. Internal Logs Overview & Demo** (NEW!)
5. [Framework Setup](#5-framework-setup)
6. [Running Tests](#6-running-tests)
7. [Quick Experiment](#7-quick-experiment)
   - **View Internal Routing Logs** (NEW!)
   - **Analyze Routing Patterns** (NEW!)
8. [**Two-Phase Routing Experiment**](#8-two-phase-routing-experiment-new)
   - **Baseline Internal Routing Behavior** (NEW!)
9. [Full Experiments](#9-full-experiments)
10. [Results Analysis](#10-results-analysis)
11. [Visualization](#11-visualization)
12. [**Internal Logs Deep Dive**](#12-internal-logs-deep-dive) (NEW!)

---

## 1. Environment Setup

Detect environment and configure accordingly

In [ ]:
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Running in Kaggle: {IN_KAGGLE}")
print(f"Python version: {sys.version}")

# Set working directory based on environment
if IN_COLAB:
    # Mount Google Drive first
    from google.colab import drive
    print("\n📁 Mounting Google Drive...")
    drive.mount('/content/drive')
    
    # Set working directory in Google Drive
    WORK_DIR = '/content/drive/MyDrive/olmoe_experiments'
    # Repository will be cloned to Google Drive (persists across sessions)
    REPO_DIR = '/content/drive/MyDrive/MOE-with-feature-selection'
else:
    # Local environment
    WORK_DIR = './olmoe_experiments'
    REPO_DIR = None

os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print(f"\n✅ Working directory: {os.getcwd()}")

# Store repo directory for later use
if IN_COLAB:
    print(f"✅ Repository location: {REPO_DIR}")

## 2. GPU Configuration

Check GPU availability and configure for optimal performance

In [4]:
import torch

# Check CUDA availability
print("=" * 70)
print("GPU CONFIGURATION")
print("=" * 70)

cuda_available = torch.cuda.is_available()
print(f"\nCUDA Available: {cuda_available}")

if cuda_available:
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    
    for i in range(torch.cuda.device_count()):
        print(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
    
    # Set device
    device = 'cuda'
    
    # Clear cache
    torch.cuda.empty_cache()
    print("\n✅ GPU is ready!")
else:
    device = 'cpu'
    print("\n⚠️  GPU not available. Using CPU (will be slower).")
    if IN_COLAB:
        print("\n💡 TIP: Enable GPU in Colab:")
        print("   Runtime → Change runtime type → Hardware accelerator → GPU")

print(f"\nDevice for experiments: {device}")
print("=" * 70)

GPU CONFIGURATION

CUDA Available: True
CUDA Version: 12.6
Number of GPUs: 1

GPU 0: NVIDIA A100-SXM4-40GB
  Memory: 42.47 GB

✅ GPU is ready!

Device for experiments: cuda


## 3. Installation

Install all required packages

In [5]:
%%bash
# Install dependencies
pip install -q torch transformers datasets pandas numpy matplotlib seaborn tqdm rich 
echo "✅ All packages installed!"

✅ All packages installed!


In [6]:
# Verify installations
import torch
import transformers
import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Package Versions:")
print(f"  torch: {torch.__version__}")
print(f"  transformers: {transformers.__version__}")
print(f"  datasets: {datasets.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")
print("\n✅ All imports successful!")

Package Versions:
  torch: 2.8.0+cu126
  transformers: 4.57.1
  datasets: 4.0.0
  pandas: 2.2.2
  numpy: 2.0.2

✅ All imports successful!


## 4. Custom Expert Selection & Model Patching

**NEW: Support for custom forward pass with internal logging**

This section implements:
1. Custom expert selection (uniform weights)
2. Model patching to return router_logits
3. Internal logging of routing decisions

In [7]:
import torch
import torch.nn.functional as F
from typing import Tuple

def custom_select_experts(
    router_logits: torch.Tensor,
    top_k: int,
    num_experts: int
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Custom expert selection with uniform weights.
    
    This is equivalent to UniformRouting but integrated directly into the model.
    
    Args:
        router_logits: [tokens, num_experts] - Raw routing scores
        top_k: Number of experts to select
        num_experts: Total number of experts
    
    Returns:
        routing_weights: [tokens, top_k] - Uniform weights (1/top_k)
        selected_experts: [tokens, top_k] - Selected expert indices
    """
    # Convert logits to probabilities
    routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
    
    # Select top-k experts based on probabilities
    _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)
    
    # KEY: Give each selected expert EQUAL probability (uniform routing)
    routing_weights = torch.ones_like(selected_experts, dtype=torch.float)
    routing_weights /= top_k
    
    return routing_weights.to(router_logits.dtype), selected_experts


def create_custom_forward(original_forward, top_k, num_experts):
    """
    Create a custom forward pass that:
    1. Uses custom_select_experts for routing
    2. Returns router_logits for analysis
    3. Enables internal logging of routing decisions
    """
    def new_forward(self, hidden_states: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        batch_size, sequence_length, hidden_dim = hidden_states.shape
        hidden_states = hidden_states.view(-1, hidden_dim)
        
        # Get router logits: (batch * sequence_length, n_experts)
        router_logits = self.gate(hidden_states)

        # Use custom expert selection
        routing_weights, selected_experts = custom_select_experts(
            router_logits,
            top_k=top_k,
            num_experts=num_experts
        )

        # Initialize output
        final_hidden_states = torch.zeros(
            (batch_size * sequence_length, hidden_dim), 
            dtype=hidden_states.dtype, 
            device=hidden_states.device
        )

        # Create expert mask for efficient indexing
        # expert_mask: [num_experts, top_k, tokens]
        expert_mask = torch.nn.functional.one_hot(
            selected_experts, 
            num_classes=num_experts
        ).permute(2, 1, 0)

        # Process each expert
        for expert_idx in range(num_experts):
            expert_layer = self.experts[expert_idx]
            
            # Get tokens assigned to this expert
            idx, top_x = torch.where(expert_mask[expert_idx])

            if top_x.numel() == 0:
                continue  # No tokens for this expert

            # Compute expert output with routing weights
            current_state = hidden_states[None, top_x].reshape(-1, hidden_dim)
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]

            # Accumulate results
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
        
        # Reshape output
        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)
        
        # Return both output and router_logits for analysis
        return final_hidden_states, router_logits
    
    return new_forward


def patch_model_with_custom_routing(model, top_k=None):
    """
    Patch OLMoE model to use custom routing with internal logging.
    
    Args:
        model: OLMoE model instance
        top_k: Number of experts to use (None = use model default)
    """
    if top_k is None:
        top_k = getattr(model.config, 'num_experts_per_tok', 8)
    
    num_experts = getattr(model.config, 'num_local_experts', 64)
    
    print(f"Patching model with custom routing:")
    print(f"  top_k: {top_k}")
    print(f"  num_experts: {num_experts}")
    
    patched_layers = 0
    
    # Patch all MoE layers
    for layer_idx, layer in enumerate(model.model.layers):
        if hasattr(layer, 'mlp') and hasattr(layer.mlp, 'experts'):
            # Save original forward
            original_forward = layer.mlp.forward
            
            # Create and apply custom forward
            layer.mlp.forward = create_custom_forward(
                original_forward, 
                top_k, 
                num_experts
            ).__get__(layer.mlp, layer.mlp.__class__)
            
            patched_layers += 1
    
    print(f"✅ Patched {patched_layers} MoE layers")
    return model


print("✅ Custom expert selection functions defined!")
print("\nFunctions available:")
print("  - custom_select_experts()")
print("  - create_custom_forward()")
print("  - patch_model_with_custom_routing()")

✅ Custom expert selection functions defined!

Functions available:
  - custom_select_experts()
  - create_custom_forward()
  - patch_model_with_custom_routing()


## 4.5. Internal Logs Overview & Demo

**What are Internal Logs?**

Internal logs capture the model's routing decisions at every layer:
- **router_logits**: Raw routing scores for each token
- **selected_experts**: Which experts were chosen
- **expert_weights**: How much weight each expert received
- **Layer-by-layer data**: Routing decisions for all 16 transformer layers

**Why are they useful?**
- Understand which experts specialize in what
- Detect routing patterns and biases
- Debug routing strategies
- Validate that modifications work correctly

**File structure:**
- `logs/{config}_{dataset}.json` - Summary metrics
- `logs/{config}_{dataset}_internal_routing.json` - Detailed routing data

In [ ]:
import torch
import torch.nn.functional as F

print("="*70)
print("INTERNAL ROUTING LOGS DEMO")
print("="*70)

# Simulate router logits for 1 token across 64 experts
torch.manual_seed(42)
router_logits = torch.randn(1, 64)

print("\n1️⃣ Router Logits (raw scores):")
print(f"   Shape: {router_logits.shape}")
print(f"   Sample values: {[f'{v:.4f}' for v in router_logits[0, :5].tolist()]}")

# Convert to probabilities
probs = F.softmax(router_logits, dim=-1)
print("\n2️⃣ Softmax Probabilities:")
print(f"   Top 5 probs: {[f'{v:.4f}' for v in torch.topk(probs, 5)[0][0].tolist()]}")
print(f"   Sum: {probs.sum().item():.6f} (should be 1.0)")

# Select top-8 experts
top_k = 8
weights, indices = torch.topk(probs, top_k)

print(f"\n3️⃣ Top-{top_k} Expert Selection:")
print(f"   Selected experts: {indices[0].tolist()}")
print(f"   Their weights: {[f'{w:.4f}' for w in weights[0].tolist()]}")
print(f"   Weight sum: {weights.sum().item():.6f}")

# Uniform routing (what custom patching does)
uniform_weights = torch.ones_like(weights) / top_k

print(f"\n4️⃣ Uniform Routing (Custom Patching):")
print(f"   Equal weights: {[f'{w:.4f}' for w in uniform_weights[0].tolist()]}")
print(f"   Each expert gets: {1/top_k:.4f}")

print("\n💡 These are the values saved in internal_routing.json files!")
print("="*70)

## 5. Framework Setup

**Clone/Update repository and load the routing experiments framework**

This will:
1. Check if repository exists in Google Drive
2. Clone if needed, or pull latest changes if it exists
3. Add repository to Python path
4. Import all framework modules

**Note:** Repository is stored in Google Drive and persists across sessions!

In [ ]:
import os
import sys

print("=" * 70)
print("FRAMEWORK SETUP")
print("=" * 70)

# Step 1: Clone repository if in Colab
if IN_COLAB:
    # Check if repo already exists in Google Drive
    if os.path.exists(REPO_DIR):
        print(f"\n📂 Step 1: Repository already exists in Google Drive")
        print(f"   Location: {REPO_DIR}")
        print(f"   ✅ Using existing repository")
        
        # Optionally pull latest changes
        print(f"\n   Pulling latest changes...")
        !cd {REPO_DIR} && git pull
    else:
        print("\n📥 Step 1: Cloning GitHub repository...")
        print(f"   Cloning to: {REPO_DIR}")
        !git clone https://github.com/aliabbasjaffri/MOE-with-feature-selection.git {REPO_DIR}
        
        # Verify clone was successful
        if os.path.exists(REPO_DIR):
            print(f"   ✅ Repository cloned successfully!")
        else:
            print(f"   ❌ Failed to clone repository!")
            raise Exception("Repository clone failed")
    
    # List main files
    print(f"\n   📂 Repository contents:")
    !ls -la {REPO_DIR}/*.py | head -5
    
    framework_dir = REPO_DIR
else:
    # Local environment - assume we're already in the repo
    framework_dir = os.path.abspath('.')
    print(f"\n📂 Step 1: Using local directory: {framework_dir}")

# Step 2: Add to Python path
print(f"\n📦 Step 2: Adding to Python path...")
if framework_dir not in sys.path:
    sys.path.insert(0, framework_dir)
    print(f"   ✅ Added: {framework_dir}")
else:
    print(f"   ✅ Already in path: {framework_dir}")

print(f"\n   Python path (first 3 entries):")
for i, p in enumerate(sys.path[:3], 1):
    print(f"     {i}. {p}")

# Step 3: Verify framework file exists
print(f"\n🔍 Step 3: Verifying framework file...")
framework_file = os.path.join(framework_dir, 'olmoe_routing_experiments.py')
if os.path.exists(framework_file):
    file_size = os.path.getsize(framework_file)
    print(f"   ✅ Found: olmoe_routing_experiments.py ({file_size:,} bytes)")
else:
    print(f"   ❌ Not found: {framework_file}")
    print(f"\n   Available Python files:")
    !ls -la {framework_dir}/*.py
    raise Exception("Framework file not found!")

print("\n" + "=" * 70)
print("✅ FRAMEWORK SETUP COMPLETE")
print("=" * 70)

In [ ]:
import sys
import importlib

print("=" * 70)
print("IMPORTING FRAMEWORK MODULES")
print("=" * 70)

# Clear any cached imports
if 'olmoe_routing_experiments' in sys.modules:
    print("🔄 Clearing cached module...")
    del sys.modules['olmoe_routing_experiments']

# Import the framework
print("\n📦 Importing olmoe_routing_experiments...")

try:
    import olmoe_routing_experiments
    print("✅ Module imported successfully!")
    
    # Import specific classes
    from olmoe_routing_experiments import (
        RoutingConfig,
        ExperimentResults,
        RoutingStrategy,
        RegularRouting,
        NormalizedRouting,
        UniformRouting,
        AdaptiveRouting,
        RoutingExperimentRunner,
        ModelPatchingUtils
    )
    
    print("\n📚 Available Components:")
    print("  • RoutingConfig, ExperimentResults")
    print("  • RoutingStrategy (base class)")
    print("  • RegularRouting, NormalizedRouting")
    print("  • UniformRouting, AdaptiveRouting")
    print("  • RoutingExperimentRunner")
    print("  • ModelPatchingUtils")
    
    print("\n" + "=" * 70)
    print("✅ ALL IMPORTS SUCCESSFUL")
    print("=" * 70)
    
except ModuleNotFoundError as e:
    print(f"\n❌ Import failed: {e}")
    print("\n🔧 Debugging info:")
    print(f"   sys.path[0]: {sys.path[0]}")
    print(f"   Current dir: {os.getcwd()}")
    
    # List Python files in path
    import glob
    py_files = glob.glob(os.path.join(sys.path[0], '*.py'))
    print(f"\n   Python files in {sys.path[0]}:")
    for f in py_files[:10]:
        print(f"     - {os.path.basename(f)}")
    
    raise

except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    import traceback
    traceback.print_exc()
    raise

### Extended Framework with Custom Patching Support

In [ ]:
class ExtendedRoutingExperimentRunner(RoutingExperimentRunner):
    """
    Extended runner with support for custom model patching.
    """
    
    def __init__(self, *args, use_custom_routing=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.use_custom_routing = use_custom_routing
        
        if use_custom_routing:
            print("\n🔧 CUSTOM ROUTING MODE ENABLED")
            print("   Model will be patched with custom_select_experts")
    
    def _set_expert_count(self, num_experts):
        """Override to support custom routing."""
        super()._set_expert_count(num_experts)
        
        if self.use_custom_routing:
            # Re-patch model with new expert count
            patch_model_with_custom_routing(self.model, top_k=num_experts)

print("✅ Extended framework with custom patching support ready!")

## 6. Running Tests

Validate the framework is working correctly

In [ ]:
print("=" * 70)
print("RUNNING VALIDATION TESTS")
print("=" * 70)

# Test 1: Routing Strategies
print("\n[1/5] Testing routing strategies...")
torch.manual_seed(42)
logits = torch.randn(1, 1, 64)

regular = RegularRouting(num_experts=8)
normalized = NormalizedRouting(num_experts=8)
uniform = UniformRouting(num_experts=8)

reg_indices, reg_weights = regular.route(logits)
norm_indices, norm_weights = normalized.route(logits)
uni_indices, uni_weights = uniform.route(logits)

# Verify uniform has equal weights
assert torch.allclose(uni_weights, torch.ones_like(uni_weights) / 8, atol=1e-6)
print("   ✅ Routing strategies work correctly")

# Test 2: Custom Expert Selection
print("\n[2/5] Testing custom expert selection...")
router_logits = torch.randn(10, 64)  # 10 tokens, 64 experts
weights, indices = custom_select_experts(router_logits, top_k=8, num_experts=64)

assert weights.shape == (10, 8)
assert indices.shape == (10, 8)
assert torch.allclose(weights, torch.ones_like(weights) / 8, atol=1e-6)
print("   ✅ Custom expert selection works correctly")

# Test 3: Different expert counts produce different routing
print("\n[3/5] Testing expert count variation...")
weights_4, indices_4 = custom_select_experts(router_logits, top_k=4, num_experts=64)
weights_16, indices_16 = custom_select_experts(router_logits, top_k=16, num_experts=64)

assert weights_4.shape[-1] == 4
assert weights_16.shape[-1] == 16
print("   ✅ Different expert counts work correctly")

# Test 4: Verify uniform weights
print("\n[4/5] Testing uniform weight distribution...")
for k in [4, 8, 16]:
    w, _ = custom_select_experts(router_logits, top_k=k, num_experts=64)
    expected = torch.ones_like(w) / k
    assert torch.allclose(w, expected, atol=1e-6)
print("   ✅ Uniform weights verified for all k values")

# Test 5: Statistics tracking
print("\n[5/5] Testing statistics tracking...")
strategy = UniformRouting(num_experts=8)
for _ in range(5):
    test_logits = torch.randn(2, 5, 64)
    strategy.route(test_logits)

stats = strategy.get_summary_stats()
assert 'avg_entropy' in stats
assert 'avg_concentration' in stats
assert 'unique_experts' in stats
print("   ✅ Statistics tracking works correctly")

print("\n" + "=" * 70)
print("✅ ALL TESTS PASSED!")
print("=" * 70)

## 7. Quick Experiment

Run a minimal experiment to verify everything works (~5 minutes)

**This experiment will:**
- Test 2 expert counts (8, 16)
- Test 2 strategies (regular, custom uniform)
- Evaluate on 50 samples
- Generate visualizations

In [ ]:
print("=" * 70)
print("QUICK EXPERIMENT (Standard Routing)")
print("=" * 70)
print("\nConfiguration:")
print("  Expert counts: [8, 16]")
print("  Strategies: [regular, uniform]")
print("  Dataset: WikiText-2")
print("  Samples: 50")
print("  Estimated time: ~5 minutes")
print("\n" + "=" * 70)

# Create runner (standard mode)
runner_standard = RoutingExperimentRunner(
    model_name="allenai/OLMoE-1B-7B-0924",
    device=device,
    output_dir="./quick_experiment_standard"
)

# Run experiments
results_df_standard = runner_standard.run_all_experiments(
    expert_counts=[8, 16],
    strategies=['regular', 'uniform'],
    datasets=['wikitext'],
    max_samples=50
)

print("\n✅ Standard routing experiment complete!")

In [ ]:
# Display results
print("\n📊 QUICK EXPERIMENT RESULTS (Standard Routing)\n")
print(results_df_standard[[
    'config', 'perplexity', 'token_accuracy', 
    'tokens_per_second', 'avg_entropy'
]].to_string(index=False))

# Best configuration
best_idx = results_df_standard['perplexity'].idxmin()
best = results_df_standard.loc[best_idx]

print("\n🏆 BEST CONFIGURATION:")
print(f"   Config: {best['config']}")
print(f"   Perplexity: {best['perplexity']:.2f}")
print(f"   Accuracy: {best['token_accuracy']:.4f}")
print(f"   Speed: {best['tokens_per_second']:.1f} tok/s")

### Quick Experiment with Custom Routing (Internal Logging)

**This uses the custom forward pass with internal router_logits logging**

### View Internal Routing Logs

See what's actually logged for each configuration

In [ ]:
import json
from pathlib import Path

print("="*70)
print("VIEWING INTERNAL ROUTING LOGS")
print("="*70)

# Find internal routing log files
logs_dir = Path("./quick_experiment_standard/logs")
internal_log_files = list(logs_dir.glob("*_internal_routing.json"))

if internal_log_files:
    print(f"\n✅ Found {len(internal_log_files)} internal routing log files:")
    for f in internal_log_files:
        print(f"   • {f.name}")
    
    # Load and display first one
    log_file = internal_log_files[0]
    print(f"\n📂 Loading: {log_file.name}")
    
    with open(log_file, 'r') as f:
        internal_logs = json.load(f)
    
    # Display summary
    print(f"\n📊 Summary:")
    print(f"   Config: {internal_logs['config']}")
    print(f"   Strategy: {internal_logs['strategy']}")
    print(f"   Num Experts: {internal_logs['num_experts']}")
    print(f"   Dataset: {internal_logs['dataset']}")
    print(f"   Total Samples: {internal_logs['summary']['total_samples']}")
    print(f"   Total Tokens: {internal_logs['summary']['total_tokens']}")
    
    # Show first sample
    if internal_logs['samples']:
        first_sample = internal_logs['samples'][0]
        print(f"\n🔍 First Sample Details:")
        print(f"   Sample ID: {first_sample['sample_id']}")
        print(f"   Num Tokens: {first_sample['num_tokens']}")
        print(f"   Num Layers: {len(first_sample['layers'])}")
        
        # Show first layer
        first_layer = first_sample['layers'][0]
        print(f"\n   Layer 0 Routing:")
        print(f"      Router logits shape: {first_layer['router_logits_shape']}")
        print(f"      Selected experts (token 0): {first_layer['selected_experts'][0][0]}")
        print(f"      Expert weights (token 0): {[f'{w:.4f}' for w in first_layer['expert_weights'][0][0]]}")
        
        print(f"\n💡 Full routing data available in: {log_file}")
else:
    print("\n⚠️  No internal routing logs found.")
    print("   Set save_internal_logs=True in evaluate_configuration()")

print("="*70)

### Analyze Routing Patterns from Internal Logs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

if internal_log_files:
    print("="*70)
    print("ANALYZING ROUTING PATTERNS")
    print("="*70)
    
    # Aggregate expert usage across all samples
    expert_usage = np.zeros(64)
    
    for sample in internal_logs['samples'][:10]:  # First 10 samples
        for layer_data in sample['layers']:
            selected_experts = layer_data['selected_experts']
            for token_experts in selected_experts:
                for token_expert_list in token_experts:
                    for expert_id in token_expert_list:
                        expert_usage[expert_id] += 1
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Expert usage histogram
    ax1 = axes[0]
    ax1.bar(range(64), expert_usage, alpha=0.7, color='steelblue')
    ax1.set_xlabel('Expert ID')
    ax1.set_ylabel('Activation Count')
    ax1.set_title(f'Expert Usage Pattern\\n({internal_logs["config"]})')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Usage distribution
    ax2 = axes[1]
    ax2.hist(expert_usage, bins=20, alpha=0.7, color='coral', edgecolor='black')
    ax2.set_xlabel('Activation Count')
    ax2.set_ylabel('Number of Experts')
    ax2.set_title('Distribution of Expert Utilization')
    ax2.axvline(expert_usage.mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {expert_usage.mean():.1f}')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('internal_logs_routing_pattern.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Statistics
    print(f"\n📊 Expert Usage Statistics:")
    print(f"   Most used expert: {expert_usage.argmax()} ({expert_usage.max():.0f} activations)")
    print(f"   Least used expert: {expert_usage.argmin()} ({expert_usage.min():.0f} activations)")
    print(f"   Average usage: {expert_usage.mean():.1f}")
    print(f"   Std deviation: {expert_usage.std():.1f}")
    print(f"   Unused experts: {(expert_usage == 0).sum()}")
    
    print("\n💡 Analysis shows which experts are preferred by the routing mechanism!")
    print("="*70)

In [ ]:
print("=" * 70)
print("QUICK EXPERIMENT (Custom Routing with Internal Logging)")
print("=" * 70)
print("\nConfiguration:")
print("  Expert counts: [8, 16]")
print("  Routing: Custom uniform (patched forward pass)")
print("  Dataset: WikiText-2")
print("  Samples: 50")
print("  Features: Internal router_logits logging")
print("\n" + "=" * 70)

# Create runner with custom routing enabled
runner_custom = ExtendedRoutingExperimentRunner(
    model_name="allenai/OLMoE-1B-7B-0924",
    device=device,
    output_dir="./quick_experiment_custom",
    use_custom_routing=True
)

# Patch the model
patch_model_with_custom_routing(runner_custom.model, top_k=8)

# Run experiments (only uniform strategy makes sense with custom routing)
results_df_custom = runner_custom.run_all_experiments(
    expert_counts=[8, 16],
    strategies=['uniform'],  # Custom routing is uniform
    datasets=['wikitext'],
    max_samples=50
)

print("\n✅ Custom routing experiment complete!")

In [ ]:
# Display results
print("\n📊 QUICK EXPERIMENT RESULTS (Custom Routing)\n")
print(results_df_custom[[
    'config', 'perplexity', 'token_accuracy', 
    'tokens_per_second', 'avg_entropy'
]].to_string(index=False))

# Compare with standard
print("\n📈 COMPARISON: Custom vs Standard Uniform Routing\n")

comparison_data = []
for expert_count in [8, 16]:
    std_row = results_df_standard[
        (results_df_standard['num_experts'] == expert_count) & 
        (results_df_standard['strategy'] == 'uniform')
    ].iloc[0]
    
    custom_row = results_df_custom[
        results_df_custom['num_experts'] == expert_count
    ].iloc[0]
    
    comparison_data.append({
        'Expert Count': expert_count,
        'Standard PPL': f"{std_row['perplexity']:.2f}",
        'Custom PPL': f"{custom_row['perplexity']:.2f}",
        'Difference': f"{custom_row['perplexity'] - std_row['perplexity']:.2f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
print("\n💡 Note: Small differences are expected due to implementation details.")

## 8. Two-Phase Routing Experiment (NEW!)

**🔬 Advanced: Analyze baseline routing, then test modifications**

This experiment uses a comprehensive 3-phase approach:
1. **Phase 1**: Analyze model's natural routing (baseline)
2. **Phase 2**: Apply modified routing strategies
3. **Phase 3**: Compare and generate detailed reports

This tells you:
- How does the model naturally route tokens to experts?
- Which experts are underutilized?
- Does custom routing improve or hurt quality?
- What's the impact of forcing uniform weights?

In [ ]:
print("=" * 70)
print("TWO-PHASE ROUTING EXPERIMENT")
print("=" * 70)
print("\nConfiguration:")
print("  Expert counts: [8, 16]")
print("  Datasets: [wikitext]")
print("  Samples: 50")
print("  Modifications: [uniform, normalized]")
print("\nPhases:")
print("  1. Baseline: Analyze model's natural routing")
print("  2. Modified: Test custom routing strategies")
print("  3. Analysis: Compare and generate report")
print("\nEstimated time: ~10 minutes")
print("=" * 70)

# Create runner
runner_two_phase = RoutingExperimentRunner(
    model_name="allenai/OLMoE-1B-7B-0924",
    device=device,
    output_dir="./two_phase_experiment"
)

# Run two-phase experiment
results_df, routing_insights = runner_two_phase.run_two_phase_experiment(
    expert_counts=[8, 16],
    datasets=['wikitext'],
    max_samples=50,
    routing_modifications=['uniform', 'normalized']
)

print("\n✅ Two-phase experiment complete!")
print(f"   Results saved to: {runner_two_phase.output_dir}")

### Phase 1: Baseline Insights

View what the model naturally does

In [ ]:
print("=" * 70)
print("PHASE 1: BASELINE ROUTING INSIGHTS")
print("=" * 70)

# Display insights
insights_data = []
for key, insight in routing_insights.items():
    insights_data.append({
        'Experts': insight['num_experts'],
        'Dataset': insight['dataset'],
        'Perplexity': f"{insight['baseline_perplexity']:.2f}",
        'Accuracy': f"{insight['baseline_accuracy']:.4f}",
        'Experts Used': f"{insight['unique_experts_used']}/64",
        'Utilization': f"{insight['expert_utilization']:.1%}",
        'Entropy': f"{insight['avg_entropy']:.3f}"
    })

insights_df = pd.DataFrame(insights_data)
print("\n")
print(insights_df.to_string(index=False))

print("\n\n📊 Key Observations:")
avg_util = np.mean([i['expert_utilization'] for i in routing_insights.values()])
avg_entropy = np.mean([i['avg_entropy'] for i in routing_insights.values()])

print(f"  • Average expert utilization: {avg_util:.1%}")
print(f"  • Average routing entropy: {avg_entropy:.3f}")

if avg_util < 0.5:
    print(f"  ⚠️  Many experts are underutilized!")
    print(f"      → Custom routing may help balance usage")

if avg_entropy < 1.0:
    print(f"  ⚠️  Routing is highly concentrated!")
    print(f"      → Model heavily favors certain experts")

### Baseline: Internal Routing Behavior Analysis

In [ ]:
print("="*70)
print("BASELINE: INTERNAL ROUTING BEHAVIOR")
print("="*70)

# Load baseline internal logs
baseline_logs_dir = Path("./two_phase_experiment/logs")
baseline_files = list(baseline_logs_dir.glob("*baseline*_internal_routing.json"))

if baseline_files:
    # Load first baseline log
    with open(baseline_files[0], 'r') as f:
        baseline_internal = json.load(f)
    
    print(f"\n📂 Analyzing: {baseline_files[0].name}\n")
    
    # Analyze weight concentration
    all_weights = []
    all_entropies = []
    
    for sample in baseline_internal['samples'][:20]:  # First 20 samples
        for layer_data in sample['layers']:
            weights = layer_data['expert_weights']
            for token_weights in weights:
                for weight_vec in token_weights:
                    all_weights.append(weight_vec)
                    # Calculate entropy
                    w = np.array(weight_vec)
                    w = w + 1e-10
                    entropy = -np.sum(w * np.log(w))
                    all_entropies.append(entropy)
    
    # Convert to numpy
    all_weights = np.array(all_weights)
    all_entropies = np.array(all_entropies)
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(16, 4))
    fig.suptitle('Baseline Routing Analysis', fontsize=14, fontweight='bold')
    
    # Weight distribution
    ax1 = axes[0]
    ax1.hist(all_weights.flatten(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.set_xlabel('Expert Weight')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Weight Distribution')
    ax1.axvline(all_weights.mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {all_weights.mean():.4f}')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Entropy distribution
    ax2 = axes[1]
    ax2.hist(all_entropies, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    ax2.set_xlabel('Routing Entropy')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Entropy Distribution')
    ax2.axvline(all_entropies.mean(), color='red', linestyle='--',
                linewidth=2, label=f'Mean: {all_entropies.mean():.3f}')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Max weight per token
    max_weights = all_weights.max(axis=1)
    ax3 = axes[2]
    ax3.hist(max_weights, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    ax3.set_xlabel('Max Weight (per token)')
    ax3.set_ylabel('Frequency')
    ax3.set_title('Weight Concentration')
    ax3.axvline(max_weights.mean(), color='red', linestyle='--',
                linewidth=2, label=f'Mean: {max_weights.mean():.3f}')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('baseline_routing_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 Baseline Routing Statistics:")
    print(f"   Average weight: {all_weights.mean():.4f}")
    print(f"   Average max weight: {max_weights.mean():.3f}")
    print(f"   Average entropy: {all_entropies.mean():.3f}")
    print(f"   Weight concentration: {(max_weights > 0.5).sum() / len(max_weights):.1%} tokens have max weight > 0.5")
    
    if max_weights.mean() > 0.5:
        print(f"\n💡 High weight concentration detected!")
        print(f"   → Model heavily favors certain experts per token")
        print(f"   → Uniform routing may help distribute load")
    
    if all_entropies.mean() < 1.0:
        print(f"\n💡 Low entropy detected!")
        print(f"   → Routing is highly deterministic")
        print(f"   → Few experts dominate")
    
    print("="*70)
else:
    print("\n⚠️  No baseline internal logs found")

### Phase 2 & 3: Modified Routing Comparison

Compare baseline vs modified routing strategies

In [ ]:
print("=" * 70)
print("PHASE 2 & 3: ROUTING COMPARISON")
print("=" * 70)

# For each configuration, show baseline vs modified
for num_experts in sorted(results_df['num_experts'].unique()):
    print(f"\n{'='*70}")
    print(f"Configuration: {num_experts} Experts")
    print(f"{'='*70}")
    
    config_df = results_df[results_df['num_experts'] == num_experts]
    
    # Create comparison table
    comparison = []
    baseline_row = config_df[config_df['strategy'] == 'baseline'].iloc[0]
    
    for _, row in config_df.iterrows():
        delta_ppl = row['perplexity'] - baseline_row['perplexity']
        delta_acc = row['token_accuracy'] - baseline_row['token_accuracy']
        
        comparison.append({
            'Strategy': row['strategy'],
            'Perplexity': f"{row['perplexity']:.2f}",
            'Δ PPL': f"{delta_ppl:+.2f}",
            'Accuracy': f"{row['token_accuracy']:.4f}",
            'Δ Acc': f"{delta_acc:+.4f}",
            'Speed': f"{row['tokens_per_second']:.1f}",
            'Entropy': f"{row['avg_entropy']:.3f}"
        })
    
    comp_df = pd.DataFrame(comparison)
    print("\n" + comp_df.to_string(index=False))
    
    # Highlight findings
    best_modified = config_df[config_df['strategy'] != 'baseline']['perplexity'].min()
    if best_modified < baseline_row['perplexity']:
        improvement = baseline_row['perplexity'] - best_modified
        best_strategy = config_df[config_df['perplexity'] == best_modified]['strategy'].iloc[0]
        print(f"\n✅ Modified routing improved! Best: {best_strategy} (−{improvement:.2f} PPL)")
    else:
        print(f"\nℹ️  Baseline routing performs best for this configuration")

### Visualize Two-Phase Results

View the generated comparison visualizations

In [ ]:
# Display the generated visualization
from IPython.display import Image, display

viz_path = "./two_phase_experiment/visualizations/two_phase_comparison.png"
print("📊 Two-Phase Experiment Visualizations:\n")
display(Image(filename=viz_path))

print("\n✅ Visualization includes:")
print("  1. Perplexity comparison (baseline vs modified)")
print("  2. Accuracy comparison")
print("  3. Delta perplexity (improvement/degradation)")
print("  4. Expert utilization (baseline)")
print("  5. Routing entropy comparison")
print("  6. Speed-quality trade-off")

### Generated Report

View the detailed markdown report

In [ ]:
# Display the generated report
from IPython.display import Markdown

report_path = "./two_phase_experiment/two_phase_report.md"

with open(report_path, 'r') as f:
    report_content = f.read()

print("📄 Full Report:\n")
display(Markdown(report_content))

print("\n\n📁 Files generated:")
print(f"  • {report_path}")
print(f"  • ./two_phase_experiment/two_phase_results.csv")
print(f"  • ./two_phase_experiment/two_phase_results.json")
print(f"  • ./two_phase_experiment/visualizations/two_phase_comparison.png")
print(f"  • ./two_phase_experiment/visualizations/two_phase_comparison.pdf")

In [ ]:
## 9. Full Experiments

Run comprehensive experiments with all configurations (~30-60 minutes)

**⚠️ WARNING: This will take significant time and GPU resources!**

In [ ]:
# Uncomment to run full experiments
RUN_FULL_EXPERIMENTS = True  # Set to True to run

if RUN_FULL_EXPERIMENTS:
    print("=" * 70)
    print("FULL EXPERIMENTS")
    print("=" * 70)
    print("\nConfiguration:")
    print("  Expert counts: [4, 8, 16, 32, 64]")
    print("  Strategies: [regular, normalized, uniform]")
    print("  Datasets: [wikitext, lambada]")
    print("  Samples: 500 per dataset")
    print("  Total experiments: 30")
    print("  Estimated time: ~60 minutes")
    print("\n" + "=" * 70)
    
    # Create runner
    runner_full = RoutingExperimentRunner(
        model_name="allenai/OLMoE-1B-7B-0924",
        device=device,
        output_dir="./full_experiments"
    )
    
    # Run experiments
    results_df_full = runner_full.run_all_experiments(
        expert_counts=[4, 8, 16, 32, 64],
        strategies=['regular', 'normalized', 'uniform'],
        datasets=['wikitext', 'lambada'],
        max_samples=500
    )
    
    # Generate visualizations
    runner_full.visualize_results(results_df_full)
    
    # Generate report
    runner_full.generate_report(results_df_full)
    
    print("\n✅ Full experiments complete!")
    print(f"   Results: {runner_full.output_dir}")
else:
    print("⏭️  Full experiments skipped (set RUN_FULL_EXPERIMENTS = True to run)")

## 10. Results Analysis

Analyze experiment results with various methods

In [ ]:
from analyze_results import ResultAnalyzer

# Use quick experiment results for analysis
analyzer = ResultAnalyzer("./quick_experiment_standard")

# Print summary
analyzer.print_summary()

## 12. Internal Logs Deep Dive

**Complete guide to accessing and using internal routing logs**

In [ ]:
print("="*70)
print("COMPLETE INTERNAL LOGS TUTORIAL")
print("="*70)

tutorial = """
📚 INTERNAL ROUTING LOGS - COMPLETE GUIDE
==========================================

1️⃣ WHAT'S SAVED:
   • logs/{config}_{dataset}.json
     - Summary metrics (perplexity, accuracy, speed)
     - Aggregate statistics (entropy, concentration)
   
   • logs/{config}_{dataset}_internal_routing.json
     - Detailed routing decisions for each sample
     - router_logits for each layer
     - selected_experts for each token
     - expert_weights for each token

2️⃣ FILE STRUCTURE:
   {
     "config": "16experts_uniform",
     "samples": [
       {
         "sample_id": 0,
         "num_tokens": 128,
         "layers": [
           {
             "layer": 0,
             "router_logits_shape": [1, 128, 64],
             "selected_experts": [[experts for token 0], ...],
             "expert_weights": [[weights for token 0], ...]
           },
           ... (16 layers total)
         ]
       },
       ... (multiple samples)
     ]
   }

3️⃣ HOW TO LOAD:
   import json
   with open('logs/16experts_uniform_wikitext_internal_routing.json', 'r') as f:
       logs = json.load(f)
   
   # Access first sample, first layer
   layer_0 = logs['samples'][0]['layers'][0]
   selected = layer_0['selected_experts']
   weights = layer_0['expert_weights']

4️⃣ WHAT TO ANALYZE:
   ✓ Which experts are most frequently selected?
   ✓ How does routing differ across layers?
   ✓ What's the weight distribution pattern?
   ✓ Are any experts never used?
   ✓ Does entropy change across layers?
   ✓ How does custom routing differ from baseline?

5️⃣ VISUALIZATION IDEAS:
   • Expert activation heatmap (layer vs expert)
   • Weight distribution histograms
   • Entropy evolution across layers
   • Per-token routing patterns
   • Expert specialization analysis

6️⃣ DEBUGGING USE CASES:
   • Verify modifications work (compare baseline vs custom)
   • Detect routing biases
   • Identify underutilized experts
   • Validate uniform weight distribution
   • Check for numerical issues
"""

print(tutorial)
print("\n✅ Internal logs provide complete visibility into routing decisions!")
print("="*70)

In [ ]:
print("="*70)
print("✅ INTERNAL LOGS CHECKLIST")
print("="*70)

checklist = [
    ("Internal routing logs enabled?", 
     "Set save_internal_logs=True in evaluate_configuration()"),
    ("Logs directory exists?",
     "Check ./experiment_*/logs/ for *_internal_routing.json files"),
    ("Loaded and viewed logs?",
     "Use json.load() to inspect routing decisions"),
    ("Analyzed expert usage?",
     "Aggregate selected_experts across samples"),
    ("Visualized routing patterns?",
     "Create heatmaps and histograms"),
    ("Compared baseline vs custom?",
     "Load both log files and compare weight distributions")
]

for i, (question, tip) in enumerate(checklist, 1):
    print(f"\n{i}. {question}")
    print(f"   💡 {tip}")

print("\n" + "="*70)
print("🎉 YOU NOW HAVE COMPLETE INTERNAL ROUTING VISIBILITY!")
print("="*70)

In [ ]:
# Compare strategies
comparison = analyzer.compare_strategies()
comparison

In [ ]:
# Find optimal configuration
optimal = analyzer.find_optimal_config(
    quality_weight=0.7,  # 70% weight on quality
    speed_weight=0.3     # 30% weight on speed
)

In [ ]:
# Analyze specific strategy
analyzer.analyze_strategy('uniform')

## 11. Visualization

Create custom visualizations from results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

# Load results
df = results_df_standard

# Create custom visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('OLMoE Routing Experiments Results', fontsize=16, fontweight='bold')

# 1. Perplexity by strategy
ax1 = axes[0, 0]
for strategy in df['strategy'].unique():
    strategy_df = df[df['strategy'] == strategy]
    ax1.plot(
        strategy_df['num_experts'], 
        strategy_df['perplexity'], 
        marker='o', 
        label=strategy,
        linewidth=2
    )
ax1.set_xlabel('Number of Experts')
ax1.set_ylabel('Perplexity (↓ better)')
ax1.set_title('Perplexity vs Expert Count')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Token accuracy by strategy
ax2 = axes[0, 1]
for strategy in df['strategy'].unique():
    strategy_df = df[df['strategy'] == strategy]
    ax2.plot(
        strategy_df['num_experts'], 
        strategy_df['token_accuracy'], 
        marker='s', 
        label=strategy,
        linewidth=2
    )
ax2.set_xlabel('Number of Experts')
ax2.set_ylabel('Token Accuracy (↑ better)')
ax2.set_title('Token Accuracy vs Expert Count')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Speed-quality trade-off
ax3 = axes[1, 0]
for strategy in df['strategy'].unique():
    strategy_df = df[df['strategy'] == strategy]
    ax3.scatter(
        strategy_df['perplexity'],
        strategy_df['tokens_per_second'],
        label=strategy,
        s=100,
        alpha=0.7
    )
ax3.set_xlabel('Perplexity (↓ better)')
ax3.set_ylabel('Tokens/Second (↑ better)')
ax3.set_title('Speed vs Quality Trade-off')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Routing entropy by strategy
ax4 = axes[1, 1]
for strategy in df['strategy'].unique():
    strategy_df = df[df['strategy'] == strategy]
    ax4.plot(
        strategy_df['num_experts'], 
        strategy_df['avg_entropy'], 
        marker='^', 
        label=strategy,
        linewidth=2
    )
ax4.set_xlabel('Number of Experts')
ax4.set_ylabel('Average Entropy')
ax4.set_title('Routing Entropy vs Expert Count')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('custom_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualizations created!")

### Compare Standard vs Custom Routing

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Standard vs Custom Routing Comparison', fontsize=14, fontweight='bold')

# Get uniform routing data from both
std_uniform = results_df_standard[results_df_standard['strategy'] == 'uniform']
custom_uniform = results_df_custom

# 1. Perplexity comparison
ax1 = axes[0]
x = range(len(std_uniform))
width = 0.35
ax1.bar([i - width/2 for i in x], std_uniform['perplexity'], width, label='Standard', alpha=0.8)
ax1.bar([i + width/2 for i in x], custom_uniform['perplexity'], width, label='Custom (Patched)', alpha=0.8)
ax1.set_xlabel('Configuration')
ax1.set_ylabel('Perplexity')
ax1.set_title('Perplexity: Standard vs Custom')
ax1.set_xticks(x)
ax1.set_xticklabels([f"{row['num_experts']} exp" for _, row in std_uniform.iterrows()])
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Entropy comparison
ax2 = axes[1]
ax2.bar([i - width/2 for i in x], std_uniform['avg_entropy'], width, label='Standard', alpha=0.8)
ax2.bar([i + width/2 for i in x], custom_uniform['avg_entropy'], width, label='Custom (Patched)', alpha=0.8)
ax2.set_xlabel('Configuration')
ax2.set_ylabel('Average Entropy')
ax2.set_title('Routing Entropy: Standard vs Custom')
ax2.set_xticks(x)
ax2.set_xticklabels([f"{row['num_experts']} exp" for _, row in std_uniform.iterrows()])
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('standard_vs_custom.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comparison visualization created!")

## Summary & Next Steps

### What We Accomplished

✅ **Environment Setup**: Configured GPU/CPU for optimal performance
✅ **Installation**: Installed all required packages
✅ **Custom Routing**: Implemented custom expert selection with internal logging
✅ **Model Patching**: Added support for patching MoE forward pass
✅ **Testing**: Validated all components work correctly
✅ **Quick Experiments**: Ran both standard and custom routing experiments
✅ **Analysis**: Analyzed results with multiple methods
✅ **Visualization**: Created comprehensive visualizations

### Key Findings

View the results above to understand:
1. How different expert counts affect quality (perplexity)
2. How routing strategies compare
3. Speed vs quality trade-offs
4. Routing entropy patterns
5. Differences between standard and custom routing

### Next Steps

1. **Run Full Experiments**: Set `RUN_FULL_EXPERIMENTS = True` for comprehensive analysis
2. **Custom Strategies**: Implement your own routing strategies
3. **More Datasets**: Test on additional datasets (PIQA, etc.)
4. **Deep Analysis**: Use the ResultAnalyzer for detailed comparisons
5. **Save Results**: Download results to Google Drive for further analysis

### Files Generated

- `quick_experiment_standard/` - Standard routing results
- `quick_experiment_custom/` - Custom routing results
- `custom_analysis.png` - Custom visualizations
- `standard_vs_custom.png` - Comparison plot

### Documentation

For more details, see:
- `QUICKSTART.md` - Quick start guide
- `ROUTING_EXPERIMENTS_README.md` - Complete documentation
- `ARCHITECTURE.md` - System architecture
- `IMPLEMENTATION_SUMMARY.md` - Technical details

---

**Happy experimenting! 🚀**